## Import Packages

In [2]:
from datetime import datetime, timedelta, timezone
import time
from pathlib import Path
import MetaTrader5 as mt5
import pandas as pd

## Configurations

In [4]:
SYMBOL = "EURUSD"
TIMEFRAME = mt5.TIMEFRAME_M5          # 5-minute candles
DAYS_PER_CHUNK = 180                  # ~6 months per request; adjust if needed
MAX_ITERS = 1000                      # safety stop
OUT_DIR = Path("raw_data")   # each year -> one CSV file
GROUP_BY_TZ = "UTC"                   # "UTC" or e.g., "Asia/Colombo"
PAUSE_SEC = 0.2                       # brief pause to let MT5 fetch missing history
MAX_EMPTY_ADVANCES = 365              # safety guard when scanning forward

LOGIN = None       
SERVER = None     
PASSWORD = None   

## Helper functions

In [6]:
def init_mt5():
    if not mt5.initialize():
        raise RuntimeError(f"MT5 initialize() failed: {mt5.last_error()}")

    if LOGIN and SERVER and PASSWORD:
        if not mt5.login(LOGIN, password=PASSWORD, server=SERVER):
            err = mt5.last_error()
            mt5.shutdown()
            raise RuntimeError(f"Login failed: {err}")

    if not mt5.symbol_select(SYMBOL, True):
        mt5.shutdown()
        raise RuntimeError(f"Failed to select symbol: {SYMBOL}")

In [7]:
def write_chunk_by_year(df: pd.DataFrame, out_dir: Path, group_by_tz: str = "UTC"):
    """
    Split given DataFrame (with UTC-aware 'time') into yearly CSVs.
    - Files are named: {SYMBOL}_M5_{YYYY}.csv
    - Data is written in chronological order per year.
    - We keep 'time' in UTC for fidelity; year grouping can use another tz if desired.
    """
    if df.empty:
        return

    # Ensure columns & dtypes
    expected_cols = ["time", "open", "high", "low", "close", "tick_volume", "spread", "real_volume"]
    df = df.loc[:, expected_cols].drop_duplicates(subset=["time"])
    df = df.sort_values("time")

    # Determine year for grouping
    if group_by_tz.upper() == "UTC":
        years = df["time"].dt.year
    else:
        # Convert to target tz only for grouping boundaries
        years = df["time"].dt.tz_convert(group_by_tz).dt.year

    for year, part in df.groupby(years):
        part = part.sort_values("time")
        out_path = out_dir / f"{SYMBOL}_M5_{year}.csv"
        header = not out_path.exists()
        part.to_csv(out_path, index=False, mode="a", header=header)

In [8]:
def find_first_bar_on_or_after(date_utc):
    """
    Return the timestamp (UTC-aware pandas Timestamp) of the first available bar
    whose open time is >= date_utc. Advances in weekly steps until it finds one.
    """
    probe = date_utc
    empties = 0
    step = timedelta(days=7)

    while empties < MAX_EMPTY_ADVANCES:
        # copy_rates_from returns bars starting from date_utc going forward
        r = mt5.copy_rates_from(SYMBOL, TIMEFRAME, probe, 1)
        if r is None:
            raise RuntimeError(f"copy_rates_from failed: {mt5.last_error()}")

        if len(r) > 0:
            first_ts = pd.to_datetime(r[0]['time'], unit='s', utc=True)
            return first_ts

        # No data yet; step forward a bit and try again
        probe += step
        empties += 1
        time.sleep(PAUSE_SEC)

    return None  # likely no history that far back with this broker/timeframe

In [9]:
def fetch_history_and_write():
    """
    Pages FORWARD in time using copy_rates_range() starting from the first
    trading bar on/after 2000-01-01 UTC, writing each chunk to per-year CSVs.
    """
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    if not mt5.initialize():
        raise SystemExit(f"initialize() failed: {mt5.last_error()}")

    # 1) Find the first available bar on/after 2000-01-01 (UTC)
    start_hint = datetime(2000, 1, 1, tzinfo=timezone.utc)
    first_bar_ts = find_first_bar_on_or_after(start_hint)
    if first_bar_ts is None:
        mt5.shutdown()
        print("No history found after 2000-01-01 for this symbol/timeframe.")
        return

    # 2) Forward-page in chunks until 'now'
    chunk = timedelta(days=DAYS_PER_CHUNK)
    end_of_range = datetime.now(timezone.utc)
    total_rows = 0
    windows = 0
    cur_start = first_bar_ts.to_pydatetime()

    while cur_start < end_of_range:
        cur_end = min(cur_start + chunk, end_of_range)

        rates = mt5.copy_rates_range(SYMBOL, TIMEFRAME, cur_start, cur_end)
        if rates is None:
            print("copy_rates_range returned None; error:", mt5.last_error())
            break

        if len(rates) == 0:
            # No bars in this window (e.g., holidays; far in the past)
            # Move forward and continue scanning.
            cur_start = cur_end + timedelta(seconds=1)
            continue

        df = pd.DataFrame(rates)
        df["time"] = pd.to_datetime(df["time"], unit="s", utc=True)

        # Write to year-split files; you said this handles duplicates internally
        write_chunk_by_year(df, OUT_DIR, GROUP_BY_TZ)

        total_rows += len(df)
        windows += 1

        # Advance start to just after the last bar we actually received
        last_ts = df["time"].iat[-1].to_pydatetime()
        cur_start = last_ts + timedelta(seconds=1)

        if windows % 5 == 0:
            print(f"Fetched {windows} windows; total rows so far: {total_rows}")

        time.sleep(PAUSE_SEC)

    mt5.shutdown()
    print(f"Done. Wrote data to: {OUT_DIR.resolve()}")


## Download data using helper functions

In [ ]:
init_mt5()
try:
    fetch_history_and_write()
finally:
    mt5.shutdown()

Fetched 5 windows; total rows so far: 182286
Fetched 10 windows; total rows so far: 366174
Fetched 15 windows; total rows so far: 549534
Fetched 20 windows; total rows so far: 729941
Fetched 25 windows; total rows so far: 912178
Fetched 30 windows; total rows so far: 1093614
Fetched 35 windows; total rows so far: 1276224
Fetched 40 windows; total rows so far: 1459858
Fetched 45 windows; total rows so far: 1643214
